In [84]:
#

import os
from PIL import Image
import numpy as np

input_image_dir = "../processed_gear_images"

out_images_x = []
out_images_y = []
# Read all images from gear_images directory
for folder in sorted(os.listdir(input_image_dir)):
    folderpath = os.path.join(input_image_dir,folder)
    print(folderpath, ':')
    for file in sorted(os.listdir(folderpath)):
        in_file = folderpath + "/" + file
        out_images_x.append(in_file)
        out_images_y.append(folder)
                
np_data_X = np.array([np.ravel(Image.open(fname)) for fname in out_images_x])



../processed_gear_images/axes :
../processed_gear_images/boots :
../processed_gear_images/carabiners :
../processed_gear_images/crampons :
../processed_gear_images/gloves :
../processed_gear_images/hardshell_jackets :
../processed_gear_images/harnesses :
../processed_gear_images/helmets :
../processed_gear_images/insulated_jackets :
../processed_gear_images/pulleys :
../processed_gear_images/rope :
../processed_gear_images/tents :


In [85]:
#SPLIT

import numpy as np
from sklearn.model_selection import train_test_split

#print(np_images)
x_train, x_test, y_train, y_test = train_test_split(np_data_X, out_images_y, test_size=.2)

print(x_train.shape, x_test.shape )
#print (len(y_train), len(y_test))

(1697, 49152) (425, 49152)


In [86]:
print(x_train)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [ 87  87  85 ...  87  87  85]
 ...
 [  0   0   0 ...   0   0   0]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [89]:
#FIT

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [90]:
#TEST

from sklearn.metrics import accuracy_score
preds = clf.predict(x_test)
print("Accuracy:", accuracy_score(y_test,preds))

Accuracy: 0.8564705882352941


In [91]:
from PIL import Image, ImageOps
 
def resize_image(image_url):
    
    response = requests.get(image_url)
    im = Image.open(BytesIO(response.content))    
    desired_size = 128
    
    old_size = im.size
    
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
 
    im = im.resize(new_size, Image.ANTIALIAS)
 
    new_im = Image.new("RGB", (desired_size, desired_size), (255,255,255))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
    
    #new_im = ImageOps.autocontrast(new_im, cutoff=0)
    new_im = ImageOps.equalize(new_im)
    
    #new_im.save(output_image_path)
    
    return new_im

In [68]:
from PIL import Image
import requests
from io import BytesIO

test_image = "https://shop.epictv.com/sites/default/files/ae42ad29e70ba8ce6b67d3bdb6ab5c6e.jpeg"

img = resize_image(test_image)

test_img =  [np.ravel(img)]

print(test_img)

test_pred = clf.predict(test_img)
print("Result:", test_pred)


[array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)]
Result: ['helmets']


In [93]:
# confusion matrix (notes updated)

from sklearn.metrics import confusion_matrix
y_true = y_test
y_pred = preds
print (len(y_true), len(y_pred))
confusion_matrix(y_true, y_pred)

cm = confusion_matrix(y_true, y_pred)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


NameError: name 'plt' is not defined